In [1]:
import pymongo
import requests
import json
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
config = {
    'mongo_uri': "mongo-uri",
    "api_key": "RIOT-API-KEY",
}

def get_region(server):
    if server in ['NA1', 'LA1', 'LA2', 'BR1']:
        return 'AMERICAS'
    elif server in ['EUN1', 'EUW1', 'TR1', 'RU']:
        return 'EUROPE'
    elif server in ['JP1', 'KR']:
        return 'ASIA'
    elif server in ['OC1', 'VN2', 'PH2', 'TH2', 'TW2', 'SG2']:
        return 'SEA'

In [3]:
client = pymongo.MongoClient(config['mongo_uri'])
db = client.myLOLDB
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [4]:
def get_match_timeline(match_id, region, db_collection, api_key=config['api_key']):
    if db_collection.find_one({'identifier': f"{match_id}_1"}):
        print(f"{match_id} already exists in the database")
        return

    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}"
    try:
        frames = requests.get(url).json()['info']['frames']
    except:
        return
    
    # Get the winning team first
    last_event = frames[-1]['events']
    assert last_event[-1]['type'] == 'GAME_END'
    winner = last_event[-1]['winningTeam']
    
    # Get players' stats in each frame
    for frame in frames:
        for participantGameId in range(1, 11):
            frame_data = {
                'timestamp': frame['timestamp'],
                'currentGold': frame['participantFrames'][str(participantGameId)]['currentGold'],
                "abilityHaste": frame['participantFrames'][str(participantGameId)]['championStats']['abilityHaste'],
                "abilityPower": frame['participantFrames'][str(participantGameId)]['championStats']['abilityPower'],
                "armor": frame['participantFrames'][str(participantGameId)]['championStats']['armor'],
                "armorPenetrationFlat": frame['participantFrames'][str(participantGameId)]['championStats']['armorPen'],
                "armorPenetrationPercent": frame['participantFrames'][str(participantGameId)]['championStats']['armorPenPercent'],
                "attackDamage": frame['participantFrames'][str(participantGameId)]['championStats']['attackDamage'],
                "attackSpeed": frame['participantFrames'][str(participantGameId)]['championStats']['attackSpeed'],
                "bonusArmorPenetrationPercent": frame['participantFrames'][str(participantGameId)]['championStats']['bonusArmorPenPercent'],
                "bonusMagicPenetrationPercent": frame['participantFrames'][str(participantGameId)]['championStats']['bonusMagicPenPercent'],
                "currentHealth": frame['participantFrames'][str(participantGameId)]['championStats']['health'],
                "healthRegenRate": frame['participantFrames'][str(participantGameId)]['championStats']['healthRegen'],
                "lifeSteal": frame['participantFrames'][str(participantGameId)]['championStats']['lifesteal'],
                "magicPenetrationFlat": frame['participantFrames'][str(participantGameId)]['championStats']['magicPen'],
                "magicPenetrationPercent": frame['participantFrames'][str(participantGameId)]['championStats']['magicPenPercent'],
                "magicResist": frame['participantFrames'][str(participantGameId)]['championStats']['magicResist'],
                "maxHealth": frame['participantFrames'][str(participantGameId)]['championStats']['healthMax'],
                "moveSpeed": frame['participantFrames'][str(participantGameId)]['championStats']['movementSpeed'],
                "omnivamp": frame['participantFrames'][str(participantGameId)]['championStats']['omnivamp'],
                "physicalVamp": frame['participantFrames'][str(participantGameId)]['championStats']['physicalVamp'],
                "resourceMax": frame['participantFrames'][str(participantGameId)]['championStats']['powerMax'],
                "resourceRegenRate": frame['participantFrames'][str(participantGameId)]['championStats']['powerRegen'],
                "resourceValue": frame['participantFrames'][str(participantGameId)]['championStats']['power'],
                "spellVamp": frame['participantFrames'][str(participantGameId)]['championStats']['spellVamp'],
                "identifier": f"{match_id}_{frame['participantFrames'][str(participantGameId)]['participantId']}"
            }

            if winner == 100:
                if participantGameId in range(1, 6):
                    frame_data['winner'] = 1
                else:
                    frame_data['winner'] = 0
            else:
                if participantGameId in range(6, 11):
                    frame_data['winner'] = 1
                else:
                    frame_data['winner'] = 0
            
            db_collection.insert_one(frame_data)
            print(f"Inserted {frame_data['identifier']}")

In [ ]:
matches = db['match'].find()
for match in matches:
    get_match_timeline(match['match_id'], get_region(match['match_id'].split('_')[0]).lower(), db['game_data_by_frame'])

In [ ]:
documents = db['game_data_by_frame'].find()

# Convert documents to a list of dictionaries
data = [doc for doc in documents]


In [8]:
# Create a DataFrame from the data
df = pd.DataFrame(data)

In [12]:
df = df.drop(columns=['_id', 'identifier', 'armorPenetrationFlat', 'bonusArmorPenetrationPercent', 'bonusMagicPenetrationPercent'])
df = TabularDataset(df)

train = df.sample(frac=0.8, random_state=200)
test = df.drop(train.index)

In [13]:
train.head()

,timestamp,currentGold,abilityHaste,abilityPower,armor,armorPenetrationPercent,attackDamage,attackSpeed,currentHealth,healthRegenRate,...,magicResist,maxHealth,moveSpeed,omnivamp,physicalVamp,resourceMax,resourceRegenRate,resourceValue,spellVamp,winner
7353,180036,763,0,0,32,0,79,116,869,8,...,30,869,560,0,0,389,17,389,0,1
11196,360092,541,0,49,38,0,70,108,800,8,...,36,979,440,0,0,564,104,260,0,0
2918,1299607,2270,0,0,73,0,201,262,2207,19,...,44,2207,381,0,0,615,22,515,0,1
43950,1440337,450,0,296,149,0,118,122,2424,28,...,49,2424,457,0,0,150,0,110,0,0
34859,60003,0,0,9,25,0,56,100,670,13,...,30,670,335,0,0,340,27,340,0,1


In [14]:
label = 'winner'
save_path = 'autogluon_live_client_model'
predictor = TabularPredictor(label=label, path=save_path).fit(train_data=train)

y_test = test[label]
test_data_nolab = test.drop(columns=[label])  # delete label column to prove we're not cheating


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "autogluon_live_client_model"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.5
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
Memory Avail:       5.37 GB / 15.34 GB (35.0%)
Disk Space Avail:   68.39 GB / 237.36 GB (28.8%)
Train Data 

In [15]:
predictor.feature_importance(test)

These features in provided data are not utilized by the predictor and will be ignored: ['abilityHaste', 'physicalVamp']
Computing feature importance via permutation shuffling for 20 features using 5000 rows with 5 shuffle sets...
	401.12s	= Expected runtime (80.22s per shuffle set)
	63.43s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
timestamp,0.10792,0.009151,0.000006,5,0.126761,0.089079
attackDamage,0.07088,0.005784,0.000005,5,0.082789,0.058971
armor,0.06032,0.003372,0.000001,5,0.067263,0.053377
abilityPower,0.05764,0.005903,0.000013,5,0.069795,0.045485
attackSpeed,0.04620,0.009212,0.000180,5,0.065168,0.027232
maxHealth,0.04140,0.007214,0.000106,5,0.056253,0.026547
resourceMax,0.04052,0.004120,0.000013,5,0.049003,0.032037
resourceRegenRate,0.03640,0.006107,0.000092,5,0.048975,0.023825
magicResist,0.03188,0.007281,0.000305,5,0.046872,0.016888
lifeSteal,0.03020,0.004320,0.000049,5,0.039094,0.021306


In [16]:
predictor.leaderboard(test, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.672928,0.6744,accuracy,3.815415,0.483817,776.433491,0.007008,0.001375,0.169581,2,True,14
1,NeuralNetTorch,0.656939,0.6540,accuracy,0.165134,0.032017,617.601583,0.165134,0.032017,617.601583,1,True,12
2,RandomForestEntr,0.654648,0.6544,accuracy,0.993474,0.109767,19.106348,0.993474,0.109767,19.106348,1,True,6
3,RandomForestGini,0.654373,0.6504,accuracy,0.986758,0.124058,16.102852,0.986758,0.124058,16.102852,1,True,5
4,ExtraTreesGini,0.649792,0.6492,accuracy,1.015718,0.123984,9.495289,1.015718,0.123984,9.495289,1,True,8
5,ExtraTreesEntr,0.649654,0.6504,accuracy,0.996067,0.131456,9.499874,0.996067,0.131456,9.499874,1,True,9
6,LightGBMLarge,0.648600,0.6580,accuracy,0.128923,0.015476,3.686376,0.128923,0.015476,3.686376,1,True,13
7,XGBoost,0.630549,0.6412,accuracy,0.066727,0.009070,1.877229,0.066727,0.009070,1.877229,1,True,11
8,LightGBM,0.626884,0.6348,accuracy,0.055036,0.008677,1.665848,0.055036,0.008677,1.665848,1,True,4
9,CatBoost,0.624777,0.6336,accuracy,0.081205,0.004517,17.506467,0.081205,0.004517,17.506467,1,True,7


In [2]:
def process_and_predict(input, predictor):
    team = ""
    for x in input['allPlayers']:
        if x['team'] == "ORDER":
            team = "blue"
        else:
            team = "red"
        
        print(f"Team {team}: {x['championName']}")
    
    timestamp = int(input['gameData']['gameTime'] * 1000)

    data = [
        timestamp,
        input['activePlayer']["currentGold"],
        input['activePlayer']['championStats']["abilityHaste"],
        input['activePlayer']['championStats']["abilityPower"],
        input['activePlayer']['championStats']["armor"],
        input['activePlayer']['championStats']["armorPenetrationFlat"],
        input['activePlayer']['championStats']["armorPenetrationPercent"],
        input['activePlayer']['championStats']["attackDamage"],
        input['activePlayer']['championStats']["attackSpeed"],
        input['activePlayer']['championStats']["bonusArmorPenetrationPercent"],
        input['activePlayer']['championStats']["bonusMagicPenetrationPercent"],
        input['activePlayer']['championStats']["currentHealth"],
        input['activePlayer']['championStats']["healthRegenRate"],
        input['activePlayer']['championStats']["lifeSteal"],
        input['activePlayer']['championStats']["magicPenetrationFlat"],
        input['activePlayer']['championStats']["magicPenetrationPercent"],
        input['activePlayer']['championStats']["magicResist"],
        input['activePlayer']['championStats']["maxHealth"],
        input['activePlayer']['championStats']["moveSpeed"],
        input['activePlayer']['championStats']["omnivamp"],
        input['activePlayer']['championStats']["physicalVamp"],
        input['activePlayer']['championStats']["resourceMax"],
        input['activePlayer']['championStats']["resourceRegenRate"],
        input['activePlayer']['championStats']['resourceValue'],
        input['activePlayer']['championStats']['spellVamp'],
    ]

    sample_df = pd.DataFrame([data], columns=['timestamp', 'currentGold', 'abilityHaste', 'abilityPower', 'armor', 'armorPenetrationFlat', 'armorPenetrationPercent', 'attackDamage', 'attackSpeed', 'bonusArmorPenetrationPercent', 'bonusMagicPenetrationPercent', 'currentHealth', 'healthRegenRate', 'lifeSteal', 'magicPenetrationFlat', 'magicPenetrationPercent', 'magicResist', 'maxHealth', 'moveSpeed', 'omnivamp', 'physicalVamp', 'resourceMax', 'resourceRegenRate', 'resourceValue', 'spellVamp'])
    sample_df = sample_df.drop(columns=['armorPenetrationFlat', 'bonusArmorPenetrationPercent', 'bonusMagicPenetrationPercent'])
    sample_df = TabularDataset(sample_df)

    prediction = predictor.predict(sample_df)
    pred_probs = predictor.predict_proba(sample_df)

    expected_result = prediction.get(0)
    if expected_result == 0:
        print(f'Expected LOSS, {pred_probs.iloc[0][0] * 100}% probable')
    else:
        print(f'Expected WIN, {pred_probs.iloc[0][1] * 100}% probable')
    
    print(f'Win/loss probability: {pred_probs.iloc[0][1] * 100}%/{pred_probs.iloc[0][0] * 100}%')

In [3]:
import time

def fetch_live_data():
    response = requests.get('https://127.0.0.1:2999/liveclientdata/allgamedata', verify=False).json()
    return response
    
predictor = TabularPredictor.load('autogluon_live_client_model')
while True:
    try:
        process_and_predict(fetch_live_data(), predictor)
    except:
        continue
    time.sleep(5)

c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Team blue: Aatrox
Team blue: Dr. Mundo
Team blue: Shen
Team blue: Zilean
Team blue: Annie
Team red: Ngộ Không
Team red: Brand
Team red: Rammus
Team red: Alistar
Team red: Graves
Expected WIN, 56.996870040893555% probable
Win/loss probability: 56.996870040893555%/43.003129959106445%


c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Team blue: Aatrox
Team blue: Dr. Mundo
Team blue: Shen
Team blue: Zilean
Team blue: Annie
Team red: Ngộ Không
Team red: Brand
Team red: Rammus
Team red: Alistar
Team red: Graves
Expected WIN, 56.076836585998535% probable
Win/loss probability: 56.076836585998535%/43.923163414001465%


c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

Team blue: Aatrox
Team blue: Dr. Mundo
Team blue: Shen
Team blue: Zilean
Team blue: Annie
Team red: Ngộ Không
Team red: Brand
Team red: Rammus
Team red: Alistar
Team red: Graves
Expected WIN, 55.406999588012695% probable
Win/loss probability: 55.406999588012695%/44.593000411987305%


c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Team blue: Aatrox
Team blue: Dr. Mundo
Team blue: Shen
Team blue: Zilean
Team blue: Annie
Team red: Ngộ Không
Team red: Brand
Team red: Rammus
Team red: Alistar
Team red: Graves
Expected WIN, 55.90074062347412% probable
Win/loss probability: 55.90074062347412%/44.09925937652588%


c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Team blue: Aatrox
Team blue: Dr. Mundo
Team blue: Shen
Team blue: Zilean
Team blue: Annie
Team red: Ngộ Không
Team red: Brand
Team red: Rammus
Team red: Alistar
Team red: Graves
Expected WIN, 55.5158793926239% probable
Win/loss probability: 55.5158793926239%/44.4841206073761%
